Voor mijn verdiepende opdracht wil ik GPT2 trainen op oncologische teksten. De data zal van Wikipedia en NCBI m.b.v Entrez Programming Utilities komen. Misschien worden er statistieken van SEER (statistieken in USA) gebruikt. Een extra doel is dat het model vragen kan beantwoorden. zolang dat niet geimplementeerd is wordt de input als seed gebruikt bv. 'Colon Cancer is'.

Hieronder volgt een voorbeeld van hoe dit eruit zou kunnen zien. Dit is een (voorgetrained) GPT2 model, gedownload van HuggingFace.

In [14]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")

prompt = "Colon cancer is"

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

gen_tokens = model.generate(
    input_ids,
    do_sample=True,
    temperature=0.9,
    max_length=100,
)
gen_text = tokenizer.batch_decode(gen_tokens)[0]
gen_text

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


"Colon cancer is also the most common type of cancer.\n\nThe median age of the first occurrence of any cancer in men in Europe was 44 years and the incidence of breast cancer increased from 17,000 to 32,000 a year between 1995 and 2004. In addition, over 50% of women infected with cancer had either lost their or a mother's breast, indicating that breast cancer in any cohort had been significantly more common in men (Table 2).\n\nHESP-I is"

Met behulp van Entrez Programming Utilities van Biopython kan ik artikelen van NCBI downloaden. \
Hieronder staat een API-call voorbeeld van [Biopython](https://biopython.org/docs/1.76/api/Bio.Entrez.html)

In [50]:
from Bio import Entrez
Entrez.email = "bram.koobs11@gmail.com"
handle = Entrez.einfo() # or esearch, efetch, ...
record = Entrez.read(handle)
handle.close()

print(record)

{'DbList': ['pubmed', 'protein', 'nuccore', 'ipg', 'nucleotide', 'structure', 'genome', 'annotinfo', 'assembly', 'bioproject', 'biosample', 'blastdbinfo', 'books', 'cdd', 'clinvar', 'gap', 'gapplus', 'grasp', 'dbvar', 'gene', 'gds', 'geoprofiles', 'medgen', 'mesh', 'nlmcatalog', 'omim', 'orgtrack', 'pmc', 'popset', 'proteinclusters', 'pcassay', 'protfam', 'pccompound', 'pcsubstance', 'seqannot', 'snp', 'sra', 'taxonomy', 'biocollections', 'gtr']}


Hieronder volgen 2 API-callers gemaakt door ChatGPT. De params, en de wijze van output heb ik hierbij aangepast.

In [82]:
import requests

base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
params = {
    "db": "pubmed",
    "term": "cancer",
    "retmax": 10,
    "retmode": "json",
}

response = requests.get(base_url, params=params)
data = response.json()
print(data)
data['esearchresult']['translationset']

{'header': {'type': 'esearch', 'version': '0.3'}, 'esearchresult': {'count': '5259042', 'retmax': '10', 'retstart': '0', 'idlist': ['39791354', '39791348', '39791347', '39791321', '39791317', '39791316', '39791315', '39791285', '39791283', '39791278'], 'translationset': [{'from': 'cancer', 'to': '"cancer\'s"[All Fields] OR "cancerated"[All Fields] OR "canceration"[All Fields] OR "cancerization"[All Fields] OR "cancerized"[All Fields] OR "cancerous"[All Fields] OR "neoplasms"[MeSH Terms] OR "neoplasms"[All Fields] OR "cancer"[All Fields] OR "cancers"[All Fields]'}], 'querytranslation': '"cancer s"[All Fields] OR "cancerated"[All Fields] OR "canceration"[All Fields] OR "cancerization"[All Fields] OR "cancerized"[All Fields] OR "cancerous"[All Fields] OR "neoplasms"[MeSH Terms] OR "neoplasms"[All Fields] OR "cancer"[All Fields] OR "cancers"[All Fields]'}}


[{'from': 'cancer',
  'to': '"cancer\'s"[All Fields] OR "cancerated"[All Fields] OR "canceration"[All Fields] OR "cancerization"[All Fields] OR "cancerized"[All Fields] OR "cancerous"[All Fields] OR "neoplasms"[MeSH Terms] OR "neoplasms"[All Fields] OR "cancer"[All Fields] OR "cancers"[All Fields]'}]

Deze API-caller geeft een lijst met UID (van artikelen) die aan mijn eisen (params) voldoen.

In [83]:
efetch_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
params = {
    "db": "pubmed",
    "id": "32719901,32719902",
    "retmode": "xml",
    "rettype": "full",
}

response = requests.get(efetch_url, params=params)
print(response.text)

<?xml version="1.0" ?>
<!DOCTYPE PubmedArticleSet PUBLIC "-//NLM//DTD PubMedArticle, 1st January 2024//EN" "https://dtd.nlm.nih.gov/ncbi/pubmed/out/pubmed_240101.dtd">
<PubmedArticleSet>
<PubmedArticle><MedlineCitation Status="MEDLINE" Owner="NLM" IndexingMethod="Automated"><PMID Version="1">32719901</PMID><DateCompleted><Year>2021</Year><Month>05</Month><Day>14</Day></DateCompleted><DateRevised><Year>2021</Year><Month>05</Month><Day>14</Day></DateRevised><Article PubModel="Print-Electronic"><Journal><ISSN IssnType="Electronic">1590-3478</ISSN><JournalIssue CitedMedium="Internet"><Volume>42</Volume><Issue>1</Issue><PubDate><Year>2021</Year><Month>Jan</Month></PubDate></JournalIssue><Title>Neurological sciences : official journal of the Italian Neurological Society and of the Italian Society of Clinical Neurophysiology</Title><ISOAbbreviation>Neurol Sci</ISOAbbreviation></Journal><ArticleTitle>Levodopa-responsive Holmes head titubation caused by midbrain cavernoma hemorrhage.</ArticleTi

Deze API-caller geeft alleen de abstract ookal is rettype:'full'. 

Hieronder staat de (tot nu toe) best versie. Deze heb ik zelf gemaakt aan de hand van door ChatGPT gemaakte code en deze [Biopython pagina](https://biopython.org/docs/1.76/api/Bio.Entrez.html).

In [ ]:
from Bio import Entrez

Entrez.email = "bram.koobs11@gmail.com"
search_handle = Entrez.esearch(db="pmc", term="cancer", retmax=5)
search_results = Entrez.read(search_handle)
uids = search_results["IdList"]

print(uids, '\n')

for uid in uids:
    fetch_handle = Entrez.efetch(db="pmc", id=uid, rettype="full", retmode="xml")
    records = fetch_handle.read()

    print(records, '\n')

['11717547', '11717538', '11717531', '11717528', '11717525']
b'<?xml version="1.0" ?>\n<!DOCTYPE pmc-articleset PUBLIC "-//NLM//DTD ARTICLE SET 2.0//EN" "https://dtd.nlm.nih.gov/ncbi/pmc/articleset/nlm-articleset-2.0.dtd">\n<pmc-articleset><article xmlns:mml="http://www.w3.org/1998/Math/MathML" xmlns:xlink="http://www.w3.org/1999/xlink" dtd-version="1.3" xml:lang="en" article-type="research-article">\n  <?properties open_access?>\n  <processing-meta base-tagset="archiving" mathml-version="3.0" table-model="xhtml" tagset-family="jats">\n    <restricted-by>pmc</restricted-by>\n  </processing-meta>\n  <front><journal-meta><journal-id journal-id-type="nlm-ta">AIMS Public Health</journal-id><journal-id journal-id-type="iso-abbrev">AIMS Public Health</journal-id><journal-id journal-id-type="publisher-id">PublicHealth</journal-id><journal-title-group><journal-title>AIMS Public Health</journal-title></journal-title-group><issn pub-type="epub">2327-8994</issn><publisher><publisher-name>AIMS Pre

Dit lijkt mij een goede manier om aan data te komen. Ik heb bij NCBI API-key aangemaakt, waardoor ik 10 artikelen per seconde kan krijgen. Zonder API-key zou dit 3 calls per seconde zijn.